In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function

In [ ]:
import os
os.mkdir("results")

In [ ]:
n_epochs = 5
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 10

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd

columns = [
    "age", "workclass", "fnlwgt", "education", "education-num", "marital-status",
    "occupation", "relationship", "race", "sex", "capital-gain", "capital-loss",
    "hours-per-week", "native-country", "income"
]

df = pd.read_csv('adult.data', header=None, names=columns, na_values=' ?', skipinitialspace=True)

df.tail()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
32560,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,>50K


In [ ]:
df["race"].nunique()

5

In [ ]:
df.dropna(inplace=True)

# Separate features and target
X = pd.get_dummies(df.drop(columns=['income']))
y = pd.get_dummies(df['income'])

In [ ]:
y

,<=50K,>50K
0,True,False
1,True,False
2,True,False
3,True,False
4,True,False
...,...,...
32556,True,False
32557,False,True
32558,True,False
32559,True,False


In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values.argmax(axis=1), dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Convert to PyTorch tensors
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values.argmax(axis=1), dtype=torch.long)

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)


In [ ]:
examples = enumerate(train_loader)
batch_idx, (example_data, example_targets) = next(examples)

In [ ]:
example_data.shape

torch.Size([64, 108])

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(108, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 2)


    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [ ]:
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

In [ ]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [ ]:
def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      torch.save(network.state_dict(), 'results/model.pth')
      torch.save(optimizer.state_dict(), 'results/optimizer.pth')

In [ ]:
def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [ ]:
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()


Test set: Avg. loss: 0.6928, Accuracy: 3051/6513 (47%)

Train Epoch: 1 [0/26048 (0%)]	Loss: 0.693445
Train Epoch: 1 [640/26048 (2%)]	Loss: 0.639743
Train Epoch: 1 [1280/26048 (5%)]	Loss: 0.610498
Train Epoch: 1 [1920/26048 (7%)]	Loss: 0.608220
Train Epoch: 1 [2560/26048 (10%)]	Loss: 0.589499
Train Epoch: 1 [3200/26048 (12%)]	Loss: 0.528879
Train Epoch: 1 [3840/26048 (15%)]	Loss: 0.502874
Train Epoch: 1 [4480/26048 (17%)]	Loss: 0.506714
Train Epoch: 1 [5120/26048 (20%)]	Loss: 0.518339
Train Epoch: 1 [5760/26048 (22%)]	Loss: 0.444780
Train Epoch: 1 [6400/26048 (25%)]	Loss: 0.560577
Train Epoch: 1 [7040/26048 (27%)]	Loss: 0.572957
Train Epoch: 1 [7680/26048 (29%)]	Loss: 0.475901
Train Epoch: 1 [8320/26048 (32%)]	Loss: 0.457284
Train Epoch: 1 [8960/26048 (34%)]	Loss: 0.481670
Train Epoch: 1 [9600/26048 (37%)]	Loss: 0.575289
Train Epoch: 1 [10240/26048 (39%)]	Loss: 0.455041
Train Epoch: 1 [10880/26048 (42%)]	Loss: 0.492493
Train Epoch: 1 [11520/26048 (44%)]	Loss: 0.463287
Train Epoch: 1 [1

Adversarial Debiasing

In [ ]:
class GRL(Function):
    @staticmethod
    def forward(ctx, x, λ):
        ctx.λ = λ
        return x

    @staticmethod
    def backward(ctx, grad_out):
        return -ctx.λ * grad_out, None


def grl(x, λ=1.0):
    return GRL.apply(x, λ)

In [ ]:
class RaceAdv(nn.Module):
    def __init__(self, hidden=16):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, hidden), nn.ReLU(),
            nn.Linear(hidden, 5)
        )

    def forward(self, logits_2):
        return self.net(logits_2)

In [ ]:
import numpy as np

In [ ]:
y

,<=50K,>50K
0,True,False
1,True,False
2,True,False
3,True,False
4,True,False
...,...,...
32556,True,False
32557,False,True
32558,True,False
32559,True,False


In [ ]:
df["race_label"] = df["race"].astype("category").cat.codes  # gives you integers like 0, 1, 2, ...

# Optional: save mapping for interpretation later
race_mapping = dict(enumerate(df["race"].astype("category").cat.categories))

# Now one-hot encode the features (excluding race)
features = pd.get_dummies(df.drop(columns=["race", "race_label"]))

# Store both features and labels for DataLoader
X = features.values.astype(np.float32)
y_income = y[">50K"].values.astype(np.int64)         # e.g. 0 or 1
y_race   = df["race_label"].values.astype(np.int64)

In [ ]:
from torch.utils.data import Dataset

class IncomeDataset(Dataset):
    def __init__(self, X, y_income, y_race):
        self.X = torch.tensor(X)
        self.y_income = torch.tensor(y_income)
        self.y_race = torch.tensor(y_race)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y_income[idx], self.y_race[idx]


In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

# First, split the raw data arrays (X, y_income, y_race)
X_train, X_test, y_inc_train, y_inc_test, y_race_train, y_race_test = train_test_split(
    X, y_income, y_race, test_size=0.2, random_state=42, stratify=y_race  # optional stratify
)

# Now wrap each split in a Dataset
train_dataset = IncomeDataset(X_train, y_inc_train, y_race_train)
test_dataset  = IncomeDataset(X_test,  y_inc_test,  y_race_test)

# Create loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=64, shuffle=False)


In [ ]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

In [ ]:
class MainNet(nn.Module):
    def __init__(self):
        super(MainNet, self).__init__()
        self.fc1 = nn.Linear(105, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 2)


    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.dropout(x, training=self.training)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [ ]:
λ_adv = 0.1
adv_steps_per_main = 1

network   = MainNet()
adversary = RaceAdv()

opt_main = torch.optim.Adam(network.parameters(),   lr=1e-3)
opt_adv  = torch.optim.Adam(adversary.parameters(), lr=1e-3)


def train(epoch):
    network.train(); adversary.train()

    for batch_idx, (x, y_income, y_race) in enumerate(train_loader):

        with torch.no_grad():
            logits_main = network(x)
        for _ in range(adv_steps_per_main):
            race_pred = adversary(logits_main.detach())
            loss_adv = F.cross_entropy(race_pred, y_race)

            opt_adv.zero_grad()
            loss_adv.backward()
            opt_adv.step()

        logits_main = network(x)
        income_loss = F.cross_entropy(logits_main, y_income)

        race_pred_main = adversary(grl(logits_main, λ_adv))
        race_loss_for_main = F.cross_entropy(race_pred_main, y_race)

        total_loss = income_loss + race_loss_for_main

        opt_main.zero_grad()
        total_loss.backward()
        opt_main.step()

        if batch_idx % log_interval == 0:
            print(f"Epoch {epoch} [{batch_idx}/{len(train_loader)}]  "
                  f"inc={income_loss.item():.3f}  "
                  f"adv={loss_adv.item():.3f}")


In [ ]:
def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():

    for data, target, _ in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

In [ ]:
n_epochs = 5

In [ ]:
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()

/usr/local/lib/python3.11/dist-packages/torch/nn/_reduction.py:51: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Avg. loss: 0.5207, Accuracy: 5070/6513 (78%)

Epoch 1 [0/407]  inc=0.546  adv=0.433
Epoch 1 [10/407]  inc=0.662  adv=0.421
Epoch 1 [20/407]  inc=0.583  adv=0.401
Epoch 1 [30/407]  inc=0.504  adv=0.501
Epoch 1 [40/407]  inc=0.524  adv=0.483
Epoch 1 [50/407]  inc=0.552  adv=0.690
Epoch 1 [60/407]  inc=0.366  adv=0.535
Epoch 1 [70/407]  inc=0.492  adv=0.586
Epoch 1 [80/407]  inc=0.430  adv=0.883
Epoch 1 [90/407]  inc=0.642  adv=0.638
Epoch 1 [100/407]  inc=0.599  adv=0.418
Epoch 1 [110/407]  inc=0.471  adv=0.552
Epoch 1 [120/407]  inc=0.437  adv=0.500
Epoch 1 [130/407]  inc=0.733  adv=0.591
Epoch 1 [140/407]  inc=0.528  adv=0.580
Epoch 1 [150/407]  inc=0.545  adv=0.399
Epoch 1 [160/407]  inc=0.447  adv=0.450
Epoch 1 [170/407]  inc=0.484  adv=0.573
Epoch 1 [180/407]  inc=0.498  adv=0.464
Epoch 1 [190/407]  inc=0.622  adv=0.748
Epoch 1 [200/407]  inc=0.680  adv=0.586
Epoch 1 [210/407]  inc=0.579  adv=0.779
Epoch 1 [220/407]  inc=0.618  adv=0.552
Epoch 1 [230/407]  inc=0.506  adv=